In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import time

from tqdm.notebook import tqdm
import extruct
import pickle

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15"
}

In [3]:
sitemap_url = 'https://www.nytimes.com/sitemaps/new/cooking.xml.gz'
response = requests.get(sitemap_url, headers=headers)
sitemap_xml = response.text
sitemap_soup = BeautifulSoup(sitemap_xml, features='xml')

gz_urls = []
loc_tags = sitemap_soup.find_all('loc')

for loc in loc_tags:
    gz_urls.append(loc.get_text()) 

display(gz_urls)
print(len(gz_urls))

['https://www.nytimes.com/sitemaps/new/cooking-2023-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2004-10.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2004-09.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2016-05.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-1988-05.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2008-08.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2008-11.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2013-06.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-10.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-01.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-1991-08.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2017-05.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2017-09.xml.gz',
 'https://www.nytimes.com/sitemaps/new/c

497


In [4]:
recipe_urls = []

for url in tqdm(gz_urls):
    response = requests.get(url, headers=headers)
    sitemap_soup = BeautifulSoup(response.text, features='xml')
    loc_tags = sitemap_soup.find_all('loc')
    for loc in loc_tags:
        recipe_urls.append(loc.get_text()) 


  0%|          | 0/497 [00:00<?, ?it/s]

In [7]:
display(recipe_urls)
len(recipe_urls)

['https://cooking.nytimes.com/recipes/1024284-turkey-sandwich-with-savory-blueberry-jam',
 'https://cooking.nytimes.com/recipes/1024283-pepperoncini-tuna-salad',
 'https://cooking.nytimes.com/recipes/1024128-tortillas-de-maiz',
 'https://cooking.nytimes.com/recipes/1024130-tetelas-de-frijol-negro-black-bean-masa-dumplings',
 'https://cooking.nytimes.com/recipes/1024129-gorditas-de-maiz',
 'https://cooking.nytimes.com/recipes/1024131-frijoles-de-olla-homestyle-black-beans',
 'https://cooking.nytimes.com/recipes/1024370-zucchini-tofu-udon',
 'https://cooking.nytimes.com/recipes/1024237-zucchini-pasta-with-crispy-capers-and-pistachios',
 'https://cooking.nytimes.com/recipes/1024369-fruit-salad',
 'https://cooking.nytimes.com/recipes/1024321-strawberry-basil-key-lime-pie',
 'https://cooking.nytimes.com/recipes/1024335-paprika-rubbed-pork-chops',
 'https://cooking.nytimes.com/recipes/1024334-turmeric-potato-salad',
 'https://cooking.nytimes.com/recipes/1024385-lemon-blackberry-shortcakes',


22821

In [6]:
with open('../data/nytc_raw_urls.pkl', 'wb') as f:
    pickle.dump(recipe_urls, f)

Quick and dirty prune?